In [ ]:
# 熵权法确定会员完成能力
# 将../data/有效会员信息数据.xlsx中的每位会员的预订任务限额作为指标1
# 将每位会员的预订任务开始时间与6:30的差值分钟数（3的倍数）作为指标2
# 若指标2不为0，则指标2更新为自己的倒数
# 若指标2为0，则指标2统一更新为常数0.5
# x_kj表示第k个会员的指标j的值

import pandas as pd
import numpy as np
import math




In [ ]:
# 读取数据
data = pd.read_excel('../data/有效会员信息数据.xlsx')
data = data.dropna()
data = data.reset_index(drop=True)
# print(data)



In [ ]:
# 计算指标1
x_11 = data['预订任务限额'].values
# print(x_11)

# 计算指标2
x_21 = data['预订任务开始时间'].values
x_22 = []
for i in range(len(x_21)):
    x_22.append((x_21[i] - 390) / 3)
x_22 = np.array(x_22)
# print(x_22)

# 更新指标2
for i in range(len(x_22)):
    if x_22[i] != 0:
        x_22[i] = 1 / x_22[i]
    else:
        x_22[i] = 0.5
# print(x_22)
